In [1]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf

In [2]:
# Check for GPU
import tensorflow as tf
try:
    from google.colab import drive
    IN_COLAB=True
except:
    IN_COLAB=False

if IN_COLAB:
    print("We're running Colab")
else:
    print(tf.config.list_physical_devices())
    print('\nCUDA GPU: ' + str(tf.test.is_gpu_available(cuda_only=True)))

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.

CUDA GPU: True


# Data Preprocessing

In [3]:
import os
os.chdir('..')

df = pd.read_csv('./daily01-ithaca/daily01-NY_Ithaca_13_E.csv', header = 0, index_col = 0)

In [4]:
df

,WBANNO,LST_DATE,CRX_VN,LONGITUDE,LATITUDE,T_DAILY_MAX,T_DAILY_MIN,T_DAILY_MEAN,T_DAILY_AVG,P_DAILY_CALC,...,SOIL_MOISTURE_5_DAILY,SOIL_MOISTURE_10_DAILY,SOIL_MOISTURE_20_DAILY,SOIL_MOISTURE_50_DAILY,SOIL_MOISTURE_100_DAILY,SOIL_TEMP_5_DAILY,SOIL_TEMP_10_DAILY,SOIL_TEMP_20_DAILY,SOIL_TEMP_50_DAILY,SOIL_TEMP_100_DAILY
0,64758,20041027,1.201,-76.25,42.44,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,64758,20041028,1.201,-76.25,42.44,12.7,-0.3,6.2,5.0,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,64758,20041029,1.201,-76.25,42.44,16.3,2.5,9.4,9.7,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64758,20041030,1.201,-76.25,42.44,17.5,10.5,14.0,14.5,1.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,64758,20041031,1.201,-76.25,42.44,17.0,9.1,13.1,12.6,0.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
304,64758,20231101,2.622,-76.25,42.44,3.5,-2.2,0.6,0.2,3.0,...,0.352,0.328,0.329,0.364,0.021,7.0,7.7,8.7,10.0,11.6
305,64758,20231102,2.622,-76.25,42.44,6.1,-3.3,1.4,1.0,0.0,...,0.352,0.323,0.325,0.357,0.021,6.4,6.9,7.8,9.3,11.3
306,64758,20231103,2.622,-76.25,42.44,12.9,1.4,7.1,7.4,0.0,...,0.345,0.319,0.320,0.327,0.019,6.9,7.0,7.7,8.9,10.9
307,64758,20231104,2.622,-76.25,42.44,13.4,5.7,9.6,9.5,0.0,...,0.338,0.313,0.312,0.315,0.024,8.1,7.9,8.3,8.9,10.6


In [5]:
Date = pd.to_datetime(df.LST_DATE, format='%Y%m%d', errors='coerce')
df['Time'] = Date

In [6]:
df.columns

Index(['WBANNO', 'LST_DATE', 'CRX_VN', 'LONGITUDE', 'LATITUDE', 'T_DAILY_MAX',
       'T_DAILY_MIN', 'T_DAILY_MEAN', 'T_DAILY_AVG', 'P_DAILY_CALC',
       'SOLARAD_DAILY', 'SUR_TEMP_DAILY_TYPE', 'SUR_TEMP_DAILY_MAX',
       'SUR_TEMP_DAILY_MIN', 'SUR_TEMP_DAILY_AVG', 'RH_DAILY_MAX',
       'RH_DAILY_MIN', 'RH_DAILY_AVG', 'SOIL_MOISTURE_5_DAILY',
       'SOIL_MOISTURE_10_DAILY', 'SOIL_MOISTURE_20_DAILY',
       'SOIL_MOISTURE_50_DAILY', 'SOIL_MOISTURE_100_DAILY',
       'SOIL_TEMP_5_DAILY', 'SOIL_TEMP_10_DAILY', 'SOIL_TEMP_20_DAILY',
       'SOIL_TEMP_50_DAILY', 'SOIL_TEMP_100_DAILY', 'Time'],
      dtype='object')

In [27]:
data = df[['T_DAILY_MAX',
       'T_DAILY_MIN', 'T_DAILY_MEAN', 'T_DAILY_AVG', 'P_DAILY_CALC',
       'SOLARAD_DAILY', 'SUR_TEMP_DAILY_MAX',
       'SUR_TEMP_DAILY_MIN', 'SUR_TEMP_DAILY_AVG']]

In [28]:
data.index = df['Time']

In [29]:
# check for N/A
data.min()

T_DAILY_MAX          -17.90
T_DAILY_MIN          -30.90
T_DAILY_MEAN         -24.40
T_DAILY_AVG          -23.20
P_DAILY_CALC           0.00
SOLARAD_DAILY          0.06
SUR_TEMP_DAILY_MAX   -16.50
SUR_TEMP_DAILY_MIN   -36.00
SUR_TEMP_DAILY_AVG   -23.60
dtype: float64

In [30]:
data

,T_DAILY_MAX,T_DAILY_MIN,T_DAILY_MEAN,T_DAILY_AVG,P_DAILY_CALC,SOLARAD_DAILY,SUR_TEMP_DAILY_MAX,SUR_TEMP_DAILY_MIN,SUR_TEMP_DAILY_AVG
Time,,,,,,,,,
2004-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-10-28,12.7,-0.3,6.2,5.0,0.0,12.67,NaN,NaN,6.5
2004-10-29,16.3,2.5,9.4,9.7,0.0,8.99,NaN,NaN,10.2
2004-10-30,17.5,10.5,14.0,14.5,1.8,4.14,NaN,NaN,13.5
2004-10-31,17.0,9.1,13.1,12.6,0.0,3.42,NaN,NaN,10.8
...,...,...,...,...,...,...,...,...,...
2023-11-01,3.5,-2.2,0.6,0.2,3.0,5.25,9.0,-4.6,0.3
2023-11-02,6.1,-3.3,1.4,1.0,0.0,6.21,12.1,-5.2,0.6
2023-11-03,12.9,1.4,7.1,7.4,0.0,10.68,13.5,-1.1,5.7


In [31]:
data.isna().sum()

T_DAILY_MAX            76
T_DAILY_MIN            76
T_DAILY_MEAN           76
T_DAILY_AVG            77
P_DAILY_CALC           47
SOLARAD_DAILY          56
SUR_TEMP_DAILY_MAX    438
SUR_TEMP_DAILY_MIN    438
SUR_TEMP_DAILY_AVG     56
dtype: int64

In [32]:
# Check data types
data.dtypes

T_DAILY_MAX           float64
T_DAILY_MIN           float64
T_DAILY_MEAN          float64
T_DAILY_AVG           float64
P_DAILY_CALC          float64
SOLARAD_DAILY         float64
SUR_TEMP_DAILY_MAX    float64
SUR_TEMP_DAILY_MIN    float64
SUR_TEMP_DAILY_AVG    float64
dtype: object

In [33]:
data.shape

(6949, 9)

In [34]:
# forward fill the missing values  
data.ffill(axis = 0, inplace = True) 

In [35]:
data

,T_DAILY_MAX,T_DAILY_MIN,T_DAILY_MEAN,T_DAILY_AVG,P_DAILY_CALC,SOLARAD_DAILY,SUR_TEMP_DAILY_MAX,SUR_TEMP_DAILY_MIN,SUR_TEMP_DAILY_AVG
Time,,,,,,,,,
2004-10-27,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2004-10-28,12.7,-0.3,6.2,5.0,0.0,12.67,NaN,NaN,6.5
2004-10-29,16.3,2.5,9.4,9.7,0.0,8.99,NaN,NaN,10.2
2004-10-30,17.5,10.5,14.0,14.5,1.8,4.14,NaN,NaN,13.5
2004-10-31,17.0,9.1,13.1,12.6,0.0,3.42,NaN,NaN,10.8
...,...,...,...,...,...,...,...,...,...
2023-11-01,3.5,-2.2,0.6,0.2,3.0,5.25,9.0,-4.6,0.3
2023-11-02,6.1,-3.3,1.4,1.0,0.0,6.21,12.1,-5.2,0.6
2023-11-03,12.9,1.4,7.1,7.4,0.0,10.68,13.5,-1.1,5.7


In [36]:
# drop NaN at the top
data.dropna(inplace = True)

In [37]:
# set target
data['target'] = data['T_DAILY_AVG']

In [38]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(data, test_size=0.2, shuffle = False)

In [39]:
from sklearn.preprocessing import MinMaxScaler
import pickle

scaler = MinMaxScaler()
scaler.fit(train)
train = scaler.transform(train)
test = scaler.transform(test)

with open('./LSTM/models/scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [40]:
train.shape

(5252, 10)

In [41]:
test.shape

(1313, 10)

In [42]:
# splitting data into sequences
def split_sequences(features, target, seq_len, forecast_len):
    X,y = list(), list()
    for i in range(len(features)):
        end_input = i + seq_len
        end_predict = end_input + forecast_len
        if end_predict > len(features)-1:
            break
        seq_x, seq_y = features[i:end_input,:], target[end_input:end_predict]
        X.append(seq_x)
        y.append(seq_y)
    return tf.convert_to_tensor(X, dtype=tf.float64), tf.convert_to_tensor(y, dtype=tf.float64)

# Define Model

In [43]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import LSTM, Dense, RNN, LSTMCell, Input, Bidirectional
from tensorflow.keras.losses import BinaryCrossentropy, MeanSquaredError
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import TensorBoard
from tensorflow.keras.utils import plot_model

class MyModel(tf.keras.Model):

    def __init__(self, input_shape, output_shape, name = 'BiLSTM-FC'):
        super().__init__(name = name)
        self.input_layer = Input(shape = input_shape, name = 'input')
        self.lstm1 = Bidirectional(LSTM(units=30, activation = 'tanh', input_shape = input_shape, return_sequences=False, name = 'lstm_1'), name = 'bilstm_1')
        self.dense1 = Dense(units=20, activation = 'relu', name = 'dense_1')
        self.dense2 = Dense(units=10, activation = 'relu', name = 'dense_2')
        self.dense3 = Dense(units = output_shape, activation = None, name = 'dense_3')
        #self.dropout = tf.keras.layers.Dropout(0.5)

    def call(self, inputs, training=False):
        x = self.lstm1(inputs)
        x = self.dense1(x)
        x = self.dense2(x)
        x = self.dense3(x)
        #if training:
        #  x = self.dropout(x, training=training)
        return x
    
    def summary(self):
        model = Model(inputs = [self.input_layer], outputs = self.call(self.input_layer), name = self.name)
        return model.summary()

# Model Training
## input length : output length = 16:4

In [44]:
# prepare sequences
seq_len = 16
forecast_len = 4
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [45]:
X_train.shape

TensorShape([5232, 16, 9])

In [46]:
y_train.shape

TensorShape([5232, 4])

In [47]:
# create model instance
model_name = 'BiLSTM-FC_16-4'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM-FC_16-4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 16, 9)]           0         
_________________________________________________________________
bilstm_1 (Bidirectional)     (None, 60)                9600      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1220      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 44        
Total params: 11,074
Trainable params: 11,074
Non-trainable params: 0
_________________________________________________________________


In [48]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

# save trained model
model.save('./LSTM/models_daily/' + model_name)

Epoch 1/30
48/48 [==============================] - 8s 27ms/step - loss: 0.0290 - mse: 0.0290 - acc: 0.2487 - val_loss: 0.0075 - val_mse: 0.0075 - val_acc: 0.2519
Epoch 2/30
48/48 [==============================] - 1s 13ms/step - loss: 0.0081 - mse: 0.0081 - acc: 0.2642 - val_loss: 0.0072 - val_mse: 0.0072 - val_acc: 0.2786
Epoch 3/30
48/48 [==============================] - 1s 13ms/step - loss: 0.0079 - mse: 0.0079 - acc: 0.2636 - val_loss: 0.0073 - val_mse: 0.0073 - val_acc: 0.2500
Epoch 4/30
48/48 [==============================] - 1s 12ms/step - loss: 0.0078 - mse: 0.0078 - acc: 0.2797 - val_loss: 0.0069 - val_mse: 0.0069 - val_acc: 0.2137
Epoch 5/30
48/48 [==============================] - 1s 12ms/step - loss: 0.0077 - mse: 0.0077 - acc: 0.2710 - val_loss: 0.0071 - val_mse: 0.0071 - val_acc: 0.2691
Epoch 6/30
48/48 [==============================] - 1s 12ms/step - loss: 0.0077 - mse: 0.0077 - acc: 0.2989 - val_loss: 0.0067 - val_mse: 0.0067 - val_acc: 0.2767
Epoch 7/30
48/48 [====

INFO:tensorflow:Assets written to: ./LSTM/models_daily/BiLSTM-FC_16-4\assets


INFO:tensorflow:Assets written to: ./LSTM/models_daily/BiLSTM-FC_16-4\assets


In [49]:
y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

In [50]:
from sklearn.metrics import mean_squared_error
print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

mean_squared_error
train set: 0.00719854475944416
test set: 0.006707222907756202


## input length : output length = 24:6

In [51]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 24
forecast_len = 6
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [52]:
# create model instance
model_name = 'BiLSTM-FC_24-6'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM-FC_24-6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 24, 9)]           0         
_________________________________________________________________
bilstm_1 (Bidirectional)     (None, 60)                9600      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1220      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 6)                 66        
Total params: 11,096
Trainable params: 11,096
Non-trainable params: 0
_________________________________________________________________


In [53]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

# save trained model
model.save('./LSTM/models_daily/' + model_name)

Epoch 1/30
47/47 [==============================] - 4s 30ms/step - loss: 0.1425 - mse: 0.1425 - acc: 0.1449 - val_loss: 0.0202 - val_mse: 0.0202 - val_acc: 0.1740
Epoch 2/30
47/47 [==============================] - 1s 14ms/step - loss: 0.0158 - mse: 0.0158 - acc: 0.1813 - val_loss: 0.0117 - val_mse: 0.0117 - val_acc: 0.1663
Epoch 3/30
47/47 [==============================] - 1s 12ms/step - loss: 0.0121 - mse: 0.0121 - acc: 0.1849 - val_loss: 0.0103 - val_mse: 0.0103 - val_acc: 0.1587
Epoch 4/30
47/47 [==============================] - 1s 12ms/step - loss: 0.0101 - mse: 0.0101 - acc: 0.1903 - val_loss: 0.0084 - val_mse: 0.0084 - val_acc: 0.1740
Epoch 5/30
47/47 [==============================] - 1s 12ms/step - loss: 0.0092 - mse: 0.0092 - acc: 0.1873 - val_loss: 0.0076 - val_mse: 0.0076 - val_acc: 0.1663
Epoch 6/30
47/47 [==============================] - 1s 12ms/step - loss: 0.0087 - mse: 0.0087 - acc: 0.1924 - val_loss: 0.0070 - val_mse: 0.0070 - val_acc: 0.1740
Epoch 7/30
47/47 [====

INFO:tensorflow:Assets written to: ./LSTM/models_daily/BiLSTM-FC_24-6\assets


INFO:tensorflow:Assets written to: ./LSTM/models_daily/BiLSTM-FC_24-6\assets


In [54]:
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

mean_squared_error
train set: 0.0076920258463276546
test set: 0.007044866752965176


## input length : output length = 32:8

In [55]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 32
forecast_len = 8
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [56]:
# create model instance
model_name = 'BiLSTM-FC_32-8'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM-FC_32-8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 32, 9)]           0         
_________________________________________________________________
bilstm_1 (Bidirectional)     (None, 60)                9600      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1220      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 8)                 88        
Total params: 11,118
Trainable params: 11,118
Non-trainable params: 0
_________________________________________________________________


In [57]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

# save trained model
model.save('./LSTM/models_daily/' + model_name)

Epoch 1/30
47/47 [==============================] - 3s 29ms/step - loss: 0.0340 - mse: 0.0340 - acc: 0.1081 - val_loss: 0.0091 - val_mse: 0.0091 - val_acc: 0.1111
Epoch 2/30
47/47 [==============================] - 1s 12ms/step - loss: 0.0091 - mse: 0.0091 - acc: 0.1160 - val_loss: 0.0085 - val_mse: 0.0085 - val_acc: 0.1398
Epoch 3/30
47/47 [==============================] - 1s 11ms/step - loss: 0.0089 - mse: 0.0089 - acc: 0.1380 - val_loss: 0.0074 - val_mse: 0.0074 - val_acc: 0.1284
Epoch 4/30
47/47 [==============================] - 1s 11ms/step - loss: 0.0088 - mse: 0.0088 - acc: 0.1375 - val_loss: 0.0074 - val_mse: 0.0074 - val_acc: 0.1513
Epoch 5/30
47/47 [==============================] - 1s 11ms/step - loss: 0.0086 - mse: 0.0086 - acc: 0.1597 - val_loss: 0.0077 - val_mse: 0.0077 - val_acc: 0.1149
Epoch 6/30
47/47 [==============================] - 1s 11ms/step - loss: 0.0089 - mse: 0.0089 - acc: 0.1557 - val_loss: 0.0074 - val_mse: 0.0074 - val_acc: 0.1111
Epoch 7/30
47/47 [====

INFO:tensorflow:Assets written to: ./LSTM/models_daily/BiLSTM-FC_32-8\assets


INFO:tensorflow:Assets written to: ./LSTM/models_daily/BiLSTM-FC_32-8\assets


In [58]:
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_squared_error(y_test, y_hat_test, sample_weight=None))

mean_squared_error
train set: 0.007952079286579455
test set: 0.008201338772749499


## input length : output length = 40:10

In [59]:
# reset memory
tf.Graph().as_default() 

# prepare sequences
seq_len = 40
forecast_len = 10
X_train, y_train = split_sequences(train[:,:-1], train[:,-1], seq_len = seq_len, forecast_len = forecast_len)
X_test, y_test = split_sequences(test[:,:-1], test[:,-1],seq_len = seq_len, forecast_len =  forecast_len)
n_features = X_train.shape[2]

In [60]:
# create model instance
model_name = 'BiLSTM-FC_40-10'
model = MyModel(input_shape = (seq_len, n_features), output_shape = (forecast_len), name = model_name)
model.summary()

Model: "BiLSTM-FC_40-10"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input (InputLayer)           [(None, 40, 9)]           0         
_________________________________________________________________
bilstm_1 (Bidirectional)     (None, 60)                9600      
_________________________________________________________________
dense_1 (Dense)              (None, 20)                1220      
_________________________________________________________________
dense_2 (Dense)              (None, 10)                210       
_________________________________________________________________
dense_3 (Dense)              (None, 10)                110       
Total params: 11,140
Trainable params: 11,140
Non-trainable params: 0
_________________________________________________________________


In [61]:
# Fit the model
model.compile(loss=MeanSquaredError(), optimizer=Adam(learning_rate = 0.01), metrics = ['mse', 'acc'])
model.fit(X_train, 
          y_train, 
          batch_size=100,
          epochs=30,
          verbose='auto',
          callbacks=None,
          validation_split=0.1,
          shuffle=True)

# save trained model
model.save('./LSTM/models_daily/' + model_name)

Epoch 1/30
47/47 [==============================] - 3s 29ms/step - loss: 0.0552 - mse: 0.0552 - acc: 0.1045 - val_loss: 0.0093 - val_mse: 0.0093 - val_acc: 0.1036
Epoch 2/30
47/47 [==============================] - 1s 12ms/step - loss: 0.0096 - mse: 0.0096 - acc: 0.1055 - val_loss: 0.0079 - val_mse: 0.0079 - val_acc: 0.1113
Epoch 3/30
47/47 [==============================] - 1s 11ms/step - loss: 0.0093 - mse: 0.0093 - acc: 0.1060 - val_loss: 0.0076 - val_mse: 0.0076 - val_acc: 0.1094
Epoch 4/30
47/47 [==============================] - 0s 10ms/step - loss: 0.0090 - mse: 0.0090 - acc: 0.1040 - val_loss: 0.0073 - val_mse: 0.0073 - val_acc: 0.0902
Epoch 5/30
47/47 [==============================] - 0s 11ms/step - loss: 0.0087 - mse: 0.0087 - acc: 0.1075 - val_loss: 0.0089 - val_mse: 0.0089 - val_acc: 0.0960
Epoch 6/30
47/47 [==============================] - 0s 10ms/step - loss: 0.0088 - mse: 0.0088 - acc: 0.1038 - val_loss: 0.0079 - val_mse: 0.0079 - val_acc: 0.1152
Epoch 7/30
47/47 [====

INFO:tensorflow:Assets written to: ./LSTM/models_daily/BiLSTM-FC_40-10\assets


INFO:tensorflow:Assets written to: ./LSTM/models_daily/BiLSTM-FC_40-10\assets


In [62]:
# evaluate the model
from sklearn.metrics import mean_squared_error

y_hat_train = model.predict(X_train)
y_hat_test = model.predict(X_test)

print('mean_squared_error')
print('train set:', mean_squared_error(y_train, y_hat_train, sample_weight=None))
print('test set:', mean_
      squared_error(y_test, y_hat_test, sample_weight=None))

mean_squared_error
train set: 0.008322059978060103
test set: 0.008384582870436105
